In [1]:
%load_ext autoreload
%autoreload 2

main_dir = '/Users/simon/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'

In [40]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_fit_and_post(time_i=42, K=4, maxvar=2):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
        
    lons_new = np.linspace(both_nc.XC.min(), both_nc.XC.max(), 60*4)
    lats_new = np.linspace(both_nc.YC.min(), both_nc.YC.max(), 60)
    # ds = both_nc # .copy(deep=True)
    ds = both_nc.interp(coords={'YC': lats_new, 'XC': lons_new})#, method='cubic')
        
    m = pcm(K=K, features=features_pcm, maxvar=maxvar, 
            timeit=True, timeit_verb=1)
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, dim='Z', inplace=True)
    m.predict_proba(ds, features=features, dim='Z', inplace=True)
    m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCM_RANK.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m

/Users/simon/pyxpcm/pyxpcm/xarray.py:38: AccessorRegistrationWarning: registration of accessor <class 'pyxpcm.xarray.pyXpcmDataSetAccessor'> under name 'pyxpcm' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  class pyXpcmDataSetAccessor:


In [41]:
ds, m = pcm_fit_and_post(K=5)
ds

  fit.1-preprocess.1-mask: 21 ms
  fit.1-preprocess.2-feature_THETA.1-ravel: 23 ms
  fit.1-preprocess.2-feature_THETA.2-interp: 42 ms
  fit.1-preprocess.2-feature_THETA.3-scale_fit: 78 ms
  fit.1-preprocess.2-feature_THETA.4-scale_transform: 33 ms
  fit.1-preprocess.2-feature_THETA.5-reduce_fit: 125 ms
  fit.1-preprocess.2-feature_THETA.6-reduce_transform: 11 ms
  fit.1-preprocess.2-feature_THETA.total: 317 ms
  fit.1-preprocess: 317 ms
  fit.1-preprocess.3-homogeniser: 2 ms
  fit.1-preprocess.2-feature_SALT.1-ravel: 20 ms
  fit.1-preprocess.2-feature_SALT.2-interp: 18 ms
  fit.1-preprocess.2-feature_SALT.3-scale_fit: 46 ms
  fit.1-preprocess.2-feature_SALT.4-scale_transform: 29 ms
  fit.1-preprocess.2-feature_SALT.5-reduce_fit: 52 ms
  fit.1-preprocess.2-feature_SALT.6-reduce_transform: 7 ms
  fit.1-preprocess.2-feature_SALT.total: 176 ms
  fit.1-preprocess: 176 ms
  fit.1-preprocess.3-homogeniser: 1 ms
  fit.1-preprocess.4-xarray: 1 ms
  fit.1-preprocess: 523 ms
  fit.fit: 323 ms
  f

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pcm_class: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_LABELS  (YC, XC) float64 nan nan nan nan nan nan ... 4.0 4.0 4.0 4.0 4.0
    PCM_POST    (pcm_class, YC, XC) float64 nan nan nan nan ... 1.0 1.0 1.0 1.0
    PCM_RANK    (pcm_class, YC, XC) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0

In [42]:
ds.THETA

<xarray.DataArray 'THETA' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91224107, 16.43401331, 16.42829744, ..., 16.73668129,
         16.96183119, 17.00106982],
        [17.10594605, 16.58357543,         nan, ..., 17.78552694,
         17.77893846, 17.15627046],
        [17.39700127, 17.41902259, 17.57648229, ..., 18.07164888,
         18.36557512, 17.36038017]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91124901, 16.43339274, 16.42752858, ..., 16.73595127,
         16.96102677, 17.00007724],
        [17.10549598, 16.58310654,         nan, ..., 17.78463126,
         17.77757912, 17.15569575],
        [17.39632988, 17.41838157, 17.57571519, ..., 18.07098448,
         18.36467935, 17.35962105]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91069974, 16.43307493, 16.42718746, ..., 16.73555945,
         16.96057151, 16.99951095],
        [17.1052305 , 16.58282718,         nan, ..., 17.78415165,
         17.77679681, 17.15536903],
        [17.39597893, 17.41795374, 17.57531644, ..., 18.07053892,
         18.36411567, 17.35921097]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.29702708, ...,  0.        ,
          1.16704883,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.6565437 ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  1.83060692,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -7

In [43]:
ds.SALT

<xarray.DataArray 'SALT' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76241635, 35.69724222, 35.69800239, ..., 35.66702453,
         35.72830786, 35.78210389],
        [35.80910656, 35.7278929 ,         nan, ..., 35.92798663,
         35.94565591, 35.82460342],
        [35.88534927, 35.83552407, 35.89943901, ..., 35.94890202,
         36.02223367, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76245369, 35.69726089, 35.69800094, ..., 35.66722655,
         35.72839551, 35.78213623],
        [35.80910656, 35.72788364,         nan, ..., 35.92803851,
         35.94566214, 35.82460342],
        [35.88534164, 35.8354741 , 35.89938586, ..., 35.948915  ,
         36.02224104, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76246604, 35.69726589, 35.69799823, ..., 35.66740227,
         35.72845882, 35.78214477],
        [35.80910274, 35.72787431,         nan, ..., 35.92806007,
         35.9456524 , 35.82460184],
        [35.8853302 , 35.83543964, 35.89935585, ..., 35.94890432,
         36.02223341, 35.87853241]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        , 12.37136438, ...,  0.        ,
         24.41002681,  0.        ],
        [ 0.        ,  0.        ,         nan, ..., 13.43770223,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ..., 34.86512099,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77

In [79]:
import gsw

def return_rho(pt_values, salt_values, lon_values, z_values):
    ct_values = gsw.conversions.CT_from_pt(salt_values, pt_values)
    # gsw.conversions.SA_from_SP(SP, p, lon, lat)
    lon_mesh, z_mesh = np.meshgrid(lon_values, z_values)
    print(z_values)
    print(lon_values)
    pressure_mesh = gsw.p_from_z(z_mesh, lon_mesh)
    pressure_values = np.zeros(np.shape(ct_values))
    for i in range(np.shape(ct_values)[2]):
        pressure_values[:, :, i] = pressure_mesh[:, :]
    rho_values = gsw.density.rho(salt_values, ct_values, pressure_values)
    print(np.shape(rho_values))
    return rho_values, ct_values, pressure_values
    
rho_values, ct_values, pressure_values = return_rho(ds.THETA.values, ds.SALT.values, ds.YC.values, ds.Z.values)

def rho_values_dataset(dataset, rho_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(rho_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, ds.XC.values]).rename('rho')
    
    da.attrs['long_name'] = 'density'
    da.attrs['units'] = 'kg m^{-3}'
    da.attrs['valid_min'] = 1000 
    da.attrs['valid_max'] = 1100
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


def ct_values_dataset(dataset, ct_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(ct_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, ds.XC.values]).rename('ct')
    
    da.attrs['long_name'] = 'conservative temperature'
    da.attrs['units'] = 'degC'
    #da.attrs['valid_min'] = -
    #da.attrs['valid_max'] = 1100
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


def pressure_values_dataset(dataset, pressure_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(pressure_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, ds.XC.values]).rename('pressure')
    
    da.attrs['long_name'] = 'pressure'
    da.attrs['units'] = 'Pa'
    #da.attrs['valid_min'] = -
    #da.attrs['valid_max'] = 1100
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


ds, da = rho_values_dataset(ds, rho_values)
ds, da = ct_values_dataset(ds, ct_values)
ds, da = pressure_values_dataset(ds, pressure_values)


[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
[-77.98265076 -77.16456035 -76.34646994 -75.52837954 -74.71028913
 -73.89219872 -73.07410832 -72.25601791 -71.4379275  -70.6198371
 -69.80174669 -68.98365629 -68.16556588 -67.34747547 -66.52938507
 -65.71129466 -64.89320425 -64.07511385 -63.25702344 -62.43893303
 -61.62084263 -60.80275222 -59.98466181 -59.16657141 -58.348481
 -57.53039059 -56.71230019 -55.89420978 -55.07611937 -54.25802897
 -53.43993856 -52.6218481

In [69]:
np.shape(ds.rho.values)

(52, 60, 240)

In [80]:
ds.to_netcdf('density_2.nc', engine='netcdf4')

In [74]:
ds.rho

<xarray.DataArray 'rho' (Z: 52, YC: 60, XC: 240)>
array([[[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.73074144, 1019.70048581, 1019.7006219 , ...,
         1019.69651067, 1019.72066766, 1019.73881143],
        [1019.74955093, 1019.71326518,           nan, ...,
         1019.80102743, 1019.80659163, 1019.75552326],
        [1019.77976419, 1019.76411775, 1019.78777838, ...,
         1019.81331308, 1019.84263723, 1019.77685987]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.74980189, 1019.71955097, 1019.71967764, ...,
         1019.71563587, 1019.73975051, 1019.75786905],
        [1019.76860561, 1019.73232111,           nan, ...,
         1019.82008329, 1019.82562253, 1019.77457456],
        [1019.79880615, 1019.78315016, 1019.8068032 , ...,
         1019.83235876, 1019.86167209, 1019.79590299]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.77238414, 1019.74213953, 1019.74226318, ...,
         1019.73828152, 1019.762354  , 1019.78044832],
        [1019.79118322, 1019.75490211,           nan, ...,
         1019.84265677, 1019.84817858, 1019.79715054],
        [1019.82137258, 1019.80571186, 1019.82936201, ...,
         1019.85492002, 1019.8842267 , 1019.81847121]],

       ...,

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1024.28566226, 1024.28566226, 1031.16845032, ...,
         1024.28566226, 1035.97888059, 1024.28566226],
        [1024.28411455, 1024.28411455,           nan, ...,
         1031.65957464, 1024.28411455, 1024.28411455],
        [1024.28259124, 1024.28259124, 1024.28259124, ...,
         1039.04208212, 1024.28259124, 1024.28259124]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1026.13995448, 1026.13995448, 1026.13995448, ...,
         1026.13995448, 1026.13995448, 1026.13995448],
        [1026.13829651, 1026.13829651,           nan, ...,
         1026.13829651, 1026.13829651, 1026.13829651],
        [1026.13666468, 1026.13666468, 1026.13666468, ...,
         1026.13666468, 1026.13666468, 1026.13666468]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1027.97923148, 1027.97923148, 1027.97923148, ...,
         1027.97923148, 1027

In [77]:
ds.ct


<xarray.DataArray 'ct' (Z: 52, YC: 60, XC: 240)>
array([[[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [3.67011177e+01, 3.66580791e+01, 3.66591429e+01, ...,
         3.66120537e+01, 3.66636873e+01, 3.67168744e+01],
        [3.67393238e+01, 3.66820770e+01,            nan, ...,
         3.68271828e+01, 3.68456057e+01, 3.67526962e+01],
        [3.68029253e+01, 3.67507821e+01, 3.68083936e+01, ...,
         3.68342879e+01, 3.68946241e+01, 3.67977740e+01]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [3.67012054e+01, 3.66581291e+01, 3.66591796e+01, ...,
         3.66122971e+01, 3.66638171e+01, 3.67169570e+01],
        [3.67393462e+01, 3.66820909e+01,            nan, ...,
         3.68272807e+01, 3.68456801e+01, 3.67527249e+01],
        [3.68029510e+01, 3.67507628e+01, 3.68083774e+01, ...,
         3.68343345e+01, 3.68946766e+01, 3.67978119e+01]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [3.67012454e+01, 3.66581500e+01, 3.66591938e+01, ...,
         3.66124967e+01, 3.66639046e+01, 3.67169939e+01],
        [3.67393556e+01, 3.66820952e+01,            nan, ...,
         3.68273267e+01, 3.68457092e+01, 3.67527396e+01],
        [3.68029568e+01, 3.67507489e+01, 3.68083666e+01, ...,
         3.68343458e+01, 3.68946970e+01, 3.67978285e+01]],

       ...,

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [1.52844796e-02, 1.52844796e-02, 1.30365579e+01, ...,
         1.52844796e-02, 2.56227143e+01, 1.52844796e-02],
        [1.52844796e-02, 1.52844796e-02,            nan, ...,
         1.41482578e+01, 1.52844796e-02, 1.52844796e-02],
        [1.52844796e-02, 1.52844796e-02, 1.52844796e-02, ...,
         3.65202188e+01, 1.52844796e-02, 1.52844796e-02]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [1.52844796e-02, 1.52844796e-02, 1.52844796e-02, ...,
         1.52844796e-02, 1.52844796e-02, 1.52844796e-02],
        [1.52844796e-02, 1.52844796e-02,            nan, ...,
         1.52844796e-02, 1.52844796e-02, 1.52844796e-02],
        [1.52844796e-02, 1.52844796e-02, 1.52844796e-02, ...,
         1.52844796e-02, 1.52844796e-02, 1.52844796e-02]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan

In [78]:
ds.pressure

<xarray.DataArray 'pressure' (Z: 52, YC: 60, XC: 240)>
array([[[2.12231676e+00, 2.12231676e+00, 2.12231676e+00, ...,
         2.12231676e+00, 2.12231676e+00, 2.12231676e+00],
        [2.12224931e+00, 2.12224931e+00, 2.12224931e+00, ...,
         2.12224931e+00, 2.12224931e+00, 2.12224931e+00],
        [2.12217773e+00, 2.12217773e+00, 2.12217773e+00, ...,
         2.12217773e+00, 2.12217773e+00, 2.12217773e+00],
        ...,
        [2.11462878e+00, 2.11462878e+00, 2.11462878e+00, ...,
         2.11462878e+00, 2.11462878e+00, 2.11462878e+00],
        [2.11448807e+00, 2.11448807e+00, 2.11448807e+00, ...,
         2.11448807e+00, 2.11448807e+00, 2.11448807e+00],
        [2.11434958e+00, 2.11434958e+00, 2.11434958e+00, ...,
         2.11434958e+00, 2.11434958e+00, 2.11434958e+00]],

       [[6.77127765e+00, 6.77127765e+00, 6.77127765e+00, ...,
         6.77127765e+00, 6.77127765e+00, 6.77127765e+00],
        [6.77106243e+00, 6.77106243e+00, 6.77106243e+00, ...,
         6.77106243e+00, 6.77106243e+00, 6.77106243e+00],
        [6.77083404e+00, 6.77083404e+00, 6.77083404e+00, ...,
         6.77083404e+00, 6.77083404e+00, 6.77083404e+00],
        ...,
        [6.74674876e+00, 6.74674876e+00, 6.74674876e+00, ...,
         6.74674876e+00, 6.74674876e+00, 6.74674876e+00],
        [6.74629982e+00, 6.74629982e+00, 6.74629982e+00, ...,
         6.74629982e+00, 6.74629982e+00, 6.74629982e+00],
        [6.74585796e+00, 6.74585796e+00, 6.74585796e+00, ...,
         6.74585796e+00, 6.74585796e+00, 6.74585796e+00]],

       [[1.22794214e+01, 1.22794214e+01, 1.22794214e+01, ...,
         1.22794214e+01, 1.22794214e+01, 1.22794214e+01],
        [1.22790311e+01, 1.22790311e+01, 1.22790311e+01, ...,
         1.22790311e+01, 1.22790311e+01, 1.22790311e+01],
        [1.22786169e+01, 1.22786169e+01, 1.22786169e+01, ...,
         1.22786169e+01, 1.22786169e+01, 1.22786169e+01],
        ...,
        [1.22349387e+01, 1.22349387e+01, 1.22349387e+01, ...,
         1.22349387e+01, 1.22349387e+01, 1.22349387e+01],
        [1.22341246e+01, 1.22341246e+01, 1.22341246e+01, ...,
         1.22341246e+01, 1.22341246e+01, 1.22341246e+01],
        [1.22333233e+01, 1.22333233e+01, 1.22333233e+01, ...,
         1.22333233e+01, 1.22333233e+01, 1.22333233e+01]],

       ...,

       [[5.11337962e+03, 5.11337962e+03, 5.11337962e+03, ...,
         5.11337962e+03, 5.11337962e+03, 5.11337962e+03],
        [5.11321533e+03, 5.11321533e+03, 5.11321533e+03, ...,
         5.11321533e+03, 5.11321533e+03, 5.11321533e+03],
        [5.11304099e+03, 5.11304099e+03, 5.11304099e+03, ...,
         5.11304099e+03, 5.11304099e+03, 5.11304099e+03],
        ...,
        [5.09465603e+03, 5.09465603e+03, 5.09465603e+03, ...,
         5.09465603e+03, 5.09465603e+03, 5.09465603e+03],
        [5.09431335e+03, 5.09431335e+03, 5.09431335e+03, ...,
         5.09431335e+03, 5.09431335e+03, 5.09431335e+03],
        [5.09397608e+03, 5.09397608e+03, 5.09397608e+03, ...,
         5.09397608e+03, 5.09397608e+03, 5.09397608e+03]],

       [[5.52749440e+03, 5.52749440e+03, 5.52749440e+03, ...,
         5.52749440e+03, 5.52749440e+03, 5.52749440e+03],
        [5.52731667e+03, 5.52731667e+03, 5.52731667e+03, ...,
         5.52731667e+03, 5.52731667e+03, 5.52731667e+03],
        [5.52712806e+03, 5.52712806e+03, 5.52712806e+03, ...,
         5.52712806e+03, 5.52712806e+03, 5.52712806e+03],
        ...,
        [5.50723850e+03, 5.50723850e+03, 5.50723850e+03, ...,
         5.50723850e+03, 5.50723850e+03, 5.50723850e+03],
        [5.50686778e+03, 5.50686778e+03, 5.50686778e+03, ...,
         5.50686778e+03, 5.50686778e+03, 5.50686778e+03],
        [5.50650291e+03, 5.50650291e+03, 5.50650291e+03, ...,
         5.50650291e+03, 5.50650291e+03, 5.50650291e+03]],

       [[5.94233050e+03, 5.94233050e+03, 5.94233050e+03, ...,
         5.94233050e+03, 5.94233050e+03, 5.94233050e+03],
        [5.94213928e+03, 5.94213928e+03, 5.94213928e+03, ...,
         5.94213928e+03, 5.94213928e+03, 5.94213928e+03],
        [5.941936

In [60]:
da

<xarray.DataArray 'rho' (Z: 52, YC: 60, XC: 240)>
array([[[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.73074144, 1019.70048581, 1019.7006219 , ...,
         1019.69651067, 1019.72066766, 1019.73881143],
        [1019.74955093, 1019.71326518,           nan, ...,
         1019.80102743, 1019.80659163, 1019.75552326],
        [1019.77976419, 1019.76411775, 1019.78777838, ...,
         1019.81331308, 1019.84263723, 1019.77685987]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.74980189, 1019.71955097, 1019.71967764, ...,
         1019.71563587, 1019.73975051, 1019.75786905],
        [1019.76860561, 1019.73232111,           nan, ...,
         1019.82008329, 1019.82562253, 1019.77457456],
        [1019.79880615, 1019.78315016, 1019.8068032 , ...,
         1019.83235876, 1019.86167209, 1019.79590299]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1019.77238414, 1019.74213953, 1019.74226318, ...,
         1019.73828152, 1019.762354  , 1019.78044832],
        [1019.79118322, 1019.75490211,           nan, ...,
         1019.84265677, 1019.84817858, 1019.79715054],
        [1019.82137258, 1019.80571186, 1019.82936201, ...,
         1019.85492002, 1019.8842267 , 1019.81847121]],

       ...,

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1024.28566226, 1024.28566226, 1031.16845032, ...,
         1024.28566226, 1035.97888059, 1024.28566226],
        [1024.28411455, 1024.28411455,           nan, ...,
         1031.65957464, 1024.28411455, 1024.28411455],
        [1024.28259124, 1024.28259124, 1024.28259124, ...,
         1039.04208212, 1024.28259124, 1024.28259124]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1026.13995448, 1026.13995448, 1026.13995448, ...,
         1026.13995448, 1026.13995448, 1026.13995448],
        [1026.13829651, 1026.13829651,           nan, ...,
         1026.13829651, 1026.13829651, 1026.13829651],
        [1026.13666468, 1026.13666468, 1026.13666468, ...,
         1026.13666468, 1026.13666468, 1026.13666468]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1027.97923148, 1027.97923148, 1027.97923148, ...,
         1027.97923148, 1027